In [110]:
import platform
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import functools
from itertools import product
import math
import csv
import re
import time
import pandas as pd
import numpy as np
import os
import shutil
import datetime
import sys
from linearmodels import PanelOLS
from linearmodels import RandomEffects
import statsmodels.formula.api as smf

In [111]:
def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

In [112]:
class wonder:

    def mcd(self,
            MCD_ICD_10_CODE_1,
            MCD_ICD_10_CODE_2,
            MCD_CAUSE_CODE_1,
            MCD_CAUSE_CODE_2,
            query_name,
            AGEG,
            by_variables
            ):

        chrome_options = webdriver.ChromeOptions()
        download_dir = self.download_dir
        chrome_options.add_experimental_option("prefs", {
            "download.default_directory": download_dir,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
            })

        d = webdriver.Chrome(chrome_options=chrome_options)
        d.implicitly_wait(30)

        base_url = "https://wonder.cdc.gov/mcd-icd10.html"

        d.get(base_url)
        accept_button = d.find_elements_by_class_name("button2")[1]
        d.execute_script("arguments[0].click();", accept_button)
        # d.execute_script("sendTab();")
        # accept_button.click()

        ct = 1
        for var in by_variables:
            if var == 'state':
                ## By State
                d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Location']/option[text()='State']".format(ct)).click()
            elif var == 'county':
                ## By County
                d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Location']/option[text()='County']".format(ct)).click()
            elif var == 'year':
                ## By Year
                d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Year and Month']/option[text()='Year']".format(ct)).click()
            elif var == 'month':
                ## By Year
                d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Year and Month']/option[text()='Month']".format(ct)).click()
            elif var == "age":
                ## By Age Groups
                d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Five-Year Age Groups']".format(ct)).click()
            elif var == "race":
                # ## By Race
                d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Race']".format(ct)).click()
            elif var == "hispanic":
                ## By Hispanic
                d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Hispanic Origin']".format(ct)).click()
            elif var == "gender" or var == "sex":
                ## By Gender
                d.find_element_by_xpath("//select[@name='B_{}']/optgroup[@label='Demographics']/option[text()='Gender']".format(ct)).click()
            elif var == '':
                ct = ct - 1

            ct = ct + 1

        # Measures
        d.find_element_by_css_selector("input#CM_31").click()  # Crude rate 95%CI
        d.find_element_by_css_selector("input#CM_32").click()  # Crude rate SE
        d.find_element_by_css_selector("input#TO_title").send_keys(query_name)  # Run name

        def double_click(element):
            actionChains = ActionChains(d)
            actionChains.double_click(element).perform()

        # Filters

        # State
        if self.STATEG is not None:
            d.find_element_by_xpath("//select[@name='F_D77.V9']").send_keys(Keys.CONTROL, 'a')
            double_click(d.find_element_by_xpath("//select[@name='F_D77.V9']/option[@value='{}']".format(self.STATEG)))
        
        # Agebox
        if self.AGEG is not None:
            d.find_element_by_xpath("//select[@name='V_D77.V51']").send_keys(Keys.CONTROL, 'a')
            double_click(d.find_element_by_xpath("//select[@name='V_D77.V51']/option[@value='{}']".format(self.AGEG)))

        # Gender
        if self.GENDERG is not None:
            d.find_element_by_xpath("//select[@name='V_D77.V7']").send_keys(Keys.CONTROL, 'a')
            double_click(d.find_element_by_xpath("//select[@name='V_D77.V7']/option[@value='{}']".format(self.GENDERG)))

        # Hispanic
        if self.HISPANICG is not None:
            d.find_element_by_xpath("//select[@name='V_D77.V17']").send_keys(Keys.CONTROL, 'a')
            double_click(d.find_element_by_xpath("//select[@name='V_D77.V17']/option[@value='{}']".format(self.HISPANICG)))

        # Race
        if self.RACEG is not None:
            d.find_element_by_xpath("//select[@name='V_D77.V8']").send_keys(Keys.CONTROL, 'a')
            double_click(d.find_element_by_xpath("//select[@name='V_D77.V8']/option[@value='{}']".format(self.RACEG)))

        if MCD_CAUSE_CODE_1 is not None:

            ############ OPTION 2
            ## Choose 113 Cause List
            # d.find_element_by_css_selector("div.finder-picks").find_element_by_xpath("//input[@id='RO_mcdD77.V15']").click()

            box = d.find_element_by_css_selector("div.finder-textarea.finder-textarea-and")

            text1 = box.find_element_by_id("TD77.V13-AND1")
            for code in MCD_CAUSE_CODE_1:
                text1.send_keys(code)
                text1.send_keys("\n")

            if MCD_CAUSE_CODE_2 is not None:
                text2 = box.find_element_by_id("TD77.V13-AND2")
                for code in MCD_CAUSE_CODE_2:
                    text2.send_keys(code)
                    text2.send_keys("\n")

        ### SUBMIT

        # input("Hit enter...")
        # d.find_element_by_css_selector("input#export-option").click()

        export = d.find_element_by_css_selector("input#export-option")
        d.execute_script("arguments[0].click();", export)
        
        total = d.find_element_by_css_selector("input#CO_show_totals")
        d.execute_script("arguments[0].click();", total)

        # d.find_element_by_xpath("//div[@class='footer-buttons']/input[@value='Send']").click()
        send = d.find_element_by_xpath("//div[@class='footer-buttons']/input[@value='Send']")
        try:
            d.execute_script("arguments[0].click();", send)
        except TimeoutException:
            time.sleep(360)
        except:
            time.sleep(360)

        not_done = True
        while not_done:
            time.sleep(2)
            if os.path.exists("{}/{}.txt".format(download_dir, query_name)):
                not_done = False

        time.sleep(3)
        old_filename = max([download_dir + "/" + f for f in os.listdir(download_dir)], key=os.path.getctime)
        filename = download_dir + "/" + "attach.csv"
        try:
            shutil.copy(os.path.join(old_filename), filename)
            os.remove(old_filename)
        except shutil.SameFileError:
            pass
        except IOError:
            time.sleep(5)
            shutil.copy(os.path.join(old_filename), filename)
            os.remove(old_filename)

        ## READ IN CSV
        def read_wonder_csv(filename):
            initial = pd.read_csv(filename, delimiter = "\t")
            if len(initial[initial[initial.columns[0]]=='---'].index) > 0:
                final =   pd.read_csv(filename, delimiter = "\t", skipfooter = len(initial.index) - initial[initial[initial.columns[0]]=='---'].index[0])
            else:
                final = initial
            return final

        self.df = read_wonder_csv(filename)
        self.df.to_csv("{}/{}.csv".format(download_dir, query_name))
        os.remove(filename)

        d.quit()


    def mcd_or_load(
            self,
            MCD_ICD_10_CODE_1,
            MCD_ICD_10_CODE_2,
            MCD_CAUSE_CODE_1,
            MCD_CAUSE_CODE_2,
            query_name = "",
            by_variables = None,
            existing_file = False,
            AGEG = None
            ):
        if not existing_file:

            self.mcd(
                    MCD_ICD_10_CODE_1 = MCD_ICD_10_CODE_1,
                    MCD_ICD_10_CODE_2 = MCD_ICD_10_CODE_2,
                    MCD_CAUSE_CODE_1 = MCD_CAUSE_CODE_1,
                    MCD_CAUSE_CODE_2 = MCD_CAUSE_CODE_2,
                    query_name = query_name,
                    by_variables = by_variables,
                    AGEG = AGEG
                    )

        else:
            self.df = pd.read_csv("{}/{}.csv".format(self.download_dir, query_name))


    def __init__(self,
                MCD_ICD_10_CODE_1=None,
                MCD_ICD_10_CODE_2=None,
                MCD_CAUSE_CODE_1 = None,
                MCD_CAUSE_CODE_2 = None,
                query_name="",
                STATEG=None,
                AGEG=None,
                GENDERG = None,
                RACEG = None,
                HISPANICG = None,
                by_variables="",
                existing_file = False,
                download_dir = True,
                just_go=False):

        self.MCD_ICD_10_CODE_1 = MCD_ICD_10_CODE_1
        self.MCD_ICD_10_CODE_2 = MCD_ICD_10_CODE_2
        self.MCD_CAUSE_CODE_1 = MCD_CAUSE_CODE_1
        self.MCD_CAUSE_CODE_2 = MCD_CAUSE_CODE_2
        self.query_name = query_name
        self.by_variables = by_variables
        self.download_dir = download_dir
        self.existing_file = existing_file
        self.STATEG=STATEG
        self.AGEG = AGEG
        self.GENDERG = GENDERG
        self.RACEG = RACEG
        self.HISPANICG = HISPANICG

        if just_go is False:

            self.mcd_or_load(
                    MCD_ICD_10_CODE_1 = self.MCD_ICD_10_CODE_1,
                    MCD_ICD_10_CODE_2 = self.MCD_ICD_10_CODE_2,
                    MCD_CAUSE_CODE_1 = self.MCD_CAUSE_CODE_1,
                    MCD_CAUSE_CODE_2 = self.MCD_CAUSE_CODE_2,
                    query_name = self.query_name,
                    by_variables = self.by_variables,
                    existing_file = self.existing_file,
                    AGEG = self.AGEG)

            

In [113]:
state_list = [
    "01",  # Alabama
    "02",  # Alaska
    "04",  # Arizona
    "05",  # Arkansas
    "06",  # California
    "08",  # Colorado
    "09",  # Connecticut
    "10",  # Delaware
    "11",  # District of Columbia
    "12",  # Florida
    "13",  # Georgia
    "15",  # Hawaii
    "16",  # Idaho
    "17",  # Illinois
    "18",  # Indiana
    "19",  # Iowa
    "20",  # Kansas
    "21",  # Kentucky
    "22",  # Louisiana
    "23",  # Maine
    "24",  # Maryland
    "25",  # Massachusetts
    "26",  # Michigan
    "27",  # Minnesota
    "28",  # Mississippi
    "29",  # Missouri
    "30",  # Montana
    "31",  # Nebraska
    "32",  # Nevada
    "33",  # New Hampshire
    "34",  # New Jersey
    "35",  # New Mexico
    "36",  # New York
    "37",  # North Carolina
    "38",  # North Dakota
    "39",  # Ohio
    "40",  # Oklahoma
    "41",  # Oregon
    "42",  # Pennsylvania
    "44",  # Rhode Island
    "45",  # South Carolina
    "46",  # South Dakota
    "47",  # Tennessee
    "48",  # Texas
    "49",  # Utah
    "50",  # Vermont
    "51",  # Virginia
    "53",  # Washington
    "54",  # West Virginia
    "55",  # Wisconsin
    "56"   # Wyoming
]

gender_list = ["M", "F"]

cause_list = [
    "A00-B99 (Certain infectious and parasitic diseases)",
    "C00-D48 (Neoplasms)",
    "D50-D89 (Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism)",
    "E00-E89 (Endocrine, nutritional and metabolic diseases)",
    "F01-F99 (Mental and behavioural disorders)",
    "G00-G98 (Diseases of the nervous system)",
    "H00-H59 (Diseases of the eye and adnexa)",
    "H60-H95 (Diseases of the ear and mastoid process)",
    "I00-I99 (Diseases of the circulatory system)",
    "J00-J98 (Diseases of the respiratory system)",
    "K00-K92 (Diseases of the digestive system)",
    "L00-L98 (Diseases of the skin and subcutaneous tissue)",
    "M00-M99 (Diseases of the musculoskeletal system and connective tissue)",
    "N00-N99 (Diseases of the genitourinary system)",
    "O00-O99 (Pregnancy, childbirth and the puerperium)",
    "P00-P96 (Certain conditions originating in the perinatal period)",
    "Q00-Q99 (Congenital malformations, deformations and chromosomal abnormalities)",
    "R00-R99 (Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified)",
    "S00-T98 (Injury, poisoning and certain other consequences of external causes)",
    "U00-U99 (Codes for special purposes)",
    "V01-Y89 (External causes of morbidity and mortality)"
]

In [ ]:
for state, gender, cause in product(state_list, gender_list, cause_list):
    wonder(

        query_name = state + gender + cause[:7],  # Designate label for query
        download_dir = "C:\\Users\\Nick\\Downloads",  # Establish download directory
        existing_file = False, # True if the data file already exists on your local computer

        # Filter by cause(s) of death
        MCD_CAUSE_CODE_1 = [cause],
        MCD_CAUSE_CODE_2 = None,

        # Group by cross-sectional variable(s)
        by_variables = ["county", "age", "year"],

        # Filter by
        STATEG = state, # state
        AGEG = None, # age group
        GENDERG = gender, # gender
        RACEG = None, # race
        HISPANICG = None, # hispanic status

    )